In [6]:
from code_tokenizer import *

# code = """
# posicao.Trigger == "DEACTIVATION" || posicao.Trigger == "POWER_LOSS"
# """

code = """
my_list = [1, 2, 3, 4, 5]
min = my_list[0]
for(i = 1; i < my_list.length; i++) {
    if(my_list[i] < min) {
        min = my_list[i]
    }
}
console.log(min)
"""

print(color_by_char_type(code))

parsed = math_layer(code)

def scan(data):
    if(type(data) == list and len(data) > 0):
        if(data[0] == 'assign'):
            variable_name = data[1][0][1]
            current_value = data[1][1]
            print(variable_name)
            print(current_value)
            print()
            return
        for item in data:
            scan(item)
scan(parsed)


my_list = [1, 2, 3, 4, 5]
min = my_list[0]
for(i = 1; i < my_list.length; i++) {
    if(my_list[i] < min) {
        min = my_list[i]
    }
}
console.log(min)

my_list
['array_define', [['number', '1'], ['number', '2'], ['number', '3'], ['number', '4'], ['number', '5']]]

min
['word', 'my_list', ['array_call', ['number', '0']]]

i
['number', '1']

min
['word', 'my_list', ['array_call', ['word', 'i']]]

